In [1]:
!pip install google-cloud-aiplatform langchain --upgrade

In [2]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

# LangChain

In [1]:
import vertexai
import langchain
from langchain.llms import VertexAI
from google.cloud import aiplatform

PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1" # @param {type:"string"}

# define project information manually if the above code didn't work
if PROJECT_ID == "(unset)":
  PROJECT_ID = "qwiklabs-gcp-01-552593a7017b" # @param {type:"string"}

print(PROJECT_ID)
vertexai.init(project=PROJECT_ID, location=LOCATION)

print(f"LangChain version: {langchain.__version__}")
print(f"Vertex AI SDK version: {aiplatform.__version__}")


qwiklabs-gcp-01-552593a7017b
LangChain version: 0.1.14
Vertex AI SDK version: 1.46.0


In [2]:
llm = VertexAI(
    model_name="code-bison@002",
    max_output_tokens=256,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=False,
)

print(llm("""Write me a function to reverse the bits of an
             integer in Objective-C
          """))

/root/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.vertexai.VertexAI` was deprecated in langchain-community 0.0.12 and will be removed in 0.2.0. An updated version of the class exists in the langchain-google-vertexai package and should be used instead. To use it run `pip install -U langchain-google-vertexai` and import as `from langchain_google_vertexai import VertexAI`.
  warn_deprecated(
/root/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


```objective-c
// Function to reverse the bits of an integer
int reverseBits(int n) {
    int reversed = 0;
    for (int i = 0; i < 32; i++) {
        if ((n & (1 << i)) != 0) {
            reversed |= (1 << (31 - i));
        }
    }
    return reversed;
}
```


# Prompt Templates

In [3]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Context: You write SQL Queries based on natural language inputs.
    Given the following database Schema: {schema}

    Write a SQL SELECT query based on the following question: {question}.
    Only return the SQL statement
    Make sure you put semicolons after each line
    """
)

schema = """
TABLE Owners (
  OwnerID STRING(36) NOT NULL,
  OwnerName STRING(MAX) NOT NULL,
  LastName STRING(MAX),
) PRIMARY KEY(OwnerID);

TABLE Pets (
  OwnerID STRING(36) NOT NULL,
  PetID STRING(MAX) NOT NULL,
  PetType STRING(MAX) NOT NULL,
  PetName STRING(MAX) NOT NULL,
  Breed STRING(MAX) NOT NULL,
  DOB DATE,
) PRIMARY KEY(OwnerID, PetID),
  INTERLEAVE IN PARENT Owners ON DELETE CASCADE;
"""

In [4]:
print(llm(prompt_template.format(schema=schema,
                                 question="How many owners are there")))

```sql
SELECT COUNT(DISTINCT OwnerID) AS total_owners
FROM Owners;
```


In [5]:
print(llm(prompt_template.format(schema=schema,
                                 question="""Show me the number of pets
                                 for each owner. Include the name of each owner""")))

```sql
SELECT Owners.OwnerName, COUNT(Pets.PetID) AS NumberOfPets
FROM Owners
LEFT JOIN Pets ON Owners.OwnerID = Pets.OwnerID
GROUP BY Owners.OwnerName;
```


In [6]:
print(llm(prompt_template.format(schema=schema,
                                 question="Show me the dogs sorted by age")))

```sql
SELECT *
FROM Pets
WHERE PetType = 'Dog'
ORDER BY DOB;
```


In [7]:
print(llm(prompt_template.format(schema=schema,
                                 question="""Show me the average age of the pets by type.
                                 Round the average age to the year""")))

```sql
SELECT
    PetType,
    ROUND(AVG(CAST(strftime('%Y', 'now') AS INTEGER) - CAST(strftime('%Y', DOB) AS INTEGER))) AS AverageAgeInYears
FROM
    Pets
GROUP BY
    PetType;
```


# Code Generation Examples

In [8]:
import vertexai
from vertexai.language_models import CodeChatModel

vertexai.init(project="qwiklabs-gcp-01-552593a7017b", location="us-central1")
chat_model = CodeChatModel.from_pretrained("codechat-bison")
parameters = {
    "candidate_count": 1,
    "max_output_tokens": 1024,
    "temperature": 0.2
}
chat = chat_model.start_chat()


## Create a Prompt template with instructions for the model

In [9]:
prompt = '''
Context: You are a Coding Chatbot. You answer coding questions.
         You generate code in Python unless you are told otherwise.
         You follow Python code best practices as defined by the
         PEP 8 – Style Guide for Python Code

Question: {0}
Code:

'''

## Code Generation

In [10]:
question = "write python codes to download daily stock prices of S&P 500."

response = chat.send_message(prompt.format(question), **parameters)
print(response.text)

 ```python
import yfinance as yf
import pandas as pd

# Get the list of S&P 500 tickers
tickers = yf.Tickers('^GSPC').tickers

# Download daily stock prices for each ticker
for ticker in tickers:
    df = yf.download(ticker, period="1d", interval="1m")
    df.to_csv(f'{ticker}.csv')
```


In [14]:
# ! pip install yfinance
# ! pip install pandas
import yfinance as yf
import pandas as pd

# Get the list of S&P 500 tickers
tickers = yf.Tickers('^GSPC').tickers
print(tickers)

# Download daily stock prices for each ticker
for ticker in tickers:
    df = yf.download(ticker, period="1d", interval="1m")
    print(df)

{'^GSPC': yfinance.Ticker object <^GSPC>}


[*********************100%%**********************]  1 of 1 completed

                                  Open         High          Low        Close  \
Datetime                                                                        
2024-04-05 09:30:00-04:00  5158.950195  5160.470215  5157.209961  5159.580078   
2024-04-05 09:31:00-04:00  5160.060059  5164.660156  5159.689941  5164.250000   
2024-04-05 09:32:00-04:00  5164.589844  5168.080078  5164.350098  5166.339844   
2024-04-05 09:33:00-04:00  5166.279785  5166.430176  5163.250000  5164.930176   
2024-04-05 09:34:00-04:00  5164.779785  5166.089844  5164.419922  5164.930176   
...                                ...          ...          ...          ...   
2024-04-05 15:55:00-04:00  5209.830078  5211.060059  5208.589844  5209.149902   
2024-04-05 15:56:00-04:00  5208.770020  5208.830078  5205.979980  5205.979980   
2024-04-05 15:57:00-04:00  5205.970215  5206.410156  5205.729980  5206.100098   
2024-04-05 15:58:00-04:00  5206.250000  5206.470215  5204.669922  5204.709961   
2024-04-05 15:59:00-04:00  5

In [15]:
question = "write python codes to download stock tickers in S&P 500 and then download daily stock prices."

response = chat.send_message(prompt.format(question), **parameters)
print(response.text)

 ```python
import yfinance as yf
import pandas as pd

# Get the list of S&P 500 tickers
tickers = yf.Tickers('^GSPC').tickers

# Download daily stock prices for each ticker
for ticker in tickers:
    df = yf.download(ticker, period="1d", interval="1m")
    df.to_csv(f'{ticker}.csv')
```


In [16]:
import yfinance as yf
import pandas as pd

# Get the list of S&P 500 tickers
tickers = yf.Tickers('^GSPC').tickers

# Download daily stock prices for each ticker
for ticker in tickers:
    print(ticker)
    df = yf.download(ticker, period="1d", interval="1m")
    df.to_csv(f'{ticker}.csv')

^GSPC


[*********************100%%**********************]  1 of 1 completed


In [17]:
question = "write python codes to download stock tickers in S&P 500 and create a python list of stock tickers."

response = chat.send_message(prompt.format(question), **parameters)
print(response.text)

 ```python
import yfinance as yf

# Get the list of S&P 500 tickers
tickers = yf.Tickers('^GSPC').tickers

# Create a Python list of stock tickers
ticker_list = []
for ticker in tickers:
    ticker_list.append(ticker)
```


In [18]:
import yfinance as yf

# Get the list of S&P 500 tickers
tickers = yf.Tickers('^GSPC').tickers

# Create a Python list of stock tickers
ticker_list = []
for ticker in tickers:
    ticker_list.append(ticker)

print(ticker_list)

['^GSPC']


In [22]:
question = "write python codes to download individuals stock tickers that is made up in  S&P 500. dont use ^GSPC. remove\n after every ticker and create a python list. also create another list with stock tickers and the exchange in which it is traded. print the total number of tickers in the list"

response = chat.send_message(prompt.format(question), **parameters)
print(response.text)

 ```python
import requests
from bs4 import BeautifulSoup

# Get the HTML content of the S&P 500 Wikipedia page
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
response = requests.get(url)
html_content = response.text

# Parse the HTML content to extract the stock tickers
soup = BeautifulSoup(html_content, 'html.parser')
table = soup.find('table', {'class': 'wikitable sortable'})
tickers = []
for row in table.find_all('tr')[1:]:
    ticker = row.find('td').text
    ticker = ticker.replace('^', '')
    tickers.append(ticker)

# Create a Python list of individual stock tickers
ticker_list = []
for ticker in tickers:
    ticker_list.append(ticker)

# Print the total number of tickers in the list
print(f'Total number of tickers: {len(ticker_list)}')
```


In [23]:
import requests
from bs4 import BeautifulSoup

# Get the HTML content of the S&P 500 Wikipedia page
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
response = requests.get(url)
html_content = response.text

# Parse the HTML content to extract the stock tickers
soup = BeautifulSoup(html_content, 'html.parser')
table = soup.find('table', {'class': 'wikitable sortable'})
tickers = []
for row in table.find_all('tr')[1:]:
    ticker = row.find('td').text
    ticker = ticker.replace('^', '')
    tickers.append(ticker)

# Create a Python list of individual stock tickers
ticker_list = []
for ticker in tickers:
    ticker_list.append(ticker)

# Print the total number of tickers in the list
print(f'Total number of tickers: {len(ticker_list)}')

Total number of tickers: 503


In [21]:
import requests
from bs4 import BeautifulSoup

# Get the HTML content of the S&P 500 Wikipedia page
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
response = requests.get(url)
html_content = response.text

# Parse the HTML content to extract the stock tickers
soup = BeautifulSoup(html_content, 'html.parser')
table = soup.find('table', {'class': 'wikitable sortable'})
tickers = []
for row in table.find_all('tr')[1:]:
    ticker = row.find('td').text
    tickers.append(ticker)

# Create a Python list of individual stock tickers
ticker_list = []
for ticker in tickers:
    ticker_list.append(ticker)
print(ticker_list)

['MMM\n', 'AOS\n', 'ABT\n', 'ABBV\n', 'ACN\n', 'ADBE\n', 'AMD\n', 'AES\n', 'AFL\n', 'A\n', 'APD\n', 'ABNB\n', 'AKAM\n', 'ALB\n', 'ARE\n', 'ALGN\n', 'ALLE\n', 'LNT\n', 'ALL\n', 'GOOGL\n', 'GOOG\n', 'MO\n', 'AMZN\n', 'AMCR\n', 'AEE\n', 'AAL\n', 'AEP\n', 'AXP\n', 'AIG\n', 'AMT\n', 'AWK\n', 'AMP\n', 'AME\n', 'AMGN\n', 'APH\n', 'ADI\n', 'ANSS\n', 'AON\n', 'APA\n', 'AAPL\n', 'AMAT\n', 'APTV\n', 'ACGL\n', 'ADM\n', 'ANET\n', 'AJG\n', 'AIZ\n', 'T\n', 'ATO\n', 'ADSK\n', 'ADP\n', 'AZO\n', 'AVB\n', 'AVY\n', 'AXON\n', 'BKR\n', 'BALL\n', 'BAC\n', 'BK\n', 'BBWI\n', 'BAX\n', 'BDX\n', 'BRK.B\n', 'BBY\n', 'BIO\n', 'TECH\n', 'BIIB\n', 'BLK\n', 'BX\n', 'BA\n', 'BKNG\n', 'BWA\n', 'BXP\n', 'BSX\n', 'BMY\n', 'AVGO\n', 'BR\n', 'BRO\n', 'BF.B\n', 'BLDR\n', 'BG\n', 'CDNS\n', 'CZR\n', 'CPT\n', 'CPB\n', 'COF\n', 'CAH\n', 'KMX\n', 'CCL\n', 'CARR\n', 'CTLT\n', 'CAT\n', 'CBOE\n', 'CBRE\n', 'CDW\n', 'CE\n', 'COR\n', 'CNC\n', 'CNP\n', 'CF\n', 'CHRW\n', 'CRL\n', 'SCHW\n', 'CHTR\n', 'CVX\n', 'CMG\n', 'CB\n', 'CHD\n', 'C

In [27]:
question = "write python codes to download individuals stock tickers that is made up in  S&P 500. dont use ^GSPC. remove\n after every ticker and create a python list. also create another list with stock tickers and the stock exchange code in which it is traded. print the total number of tickers in the list"

response = chat.send_message(prompt.format(question), **parameters)
print(response.text)

 ```python
import requests
from bs4 import BeautifulSoup

# Get the HTML content of the S&P 500 Wikipedia page
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
response = requests.get(url)
html_content = response.text

# Parse the HTML content to extract the stock tickers and stock exchange codes
soup = BeautifulSoup(html_content, 'html.parser')
table = soup.find('table', {'class': 'wikitable sortable'})
tickers = []
stock_exchange_codes = []
for row in table.find_all('tr')[1:]:
    ticker = row.find('td').text
    ticker = ticker.replace('^', '')
    tickers.append(ticker)
    stock_exchange_code = row.find_all('td')[2].text
    stock_exchange_codes.append(stock_exchange_code)

# Create a Python list of individual stock tickers
ticker_list = []
for ticker in tickers:
    ticker_list.append(ticker)

# Create a Python list of stock tickers and stock exchange codes
ticker_stock_exchange_code_list = []
for i in range(len(tickers)):
    ticker_stock_exchange_code_list.appe

In [ ]:
question = '''Write a Python class that can be used to convert Fahrenheit to Celsius
              and visa versa
'''

response = chat.send_message(prompt.format(question), **parameters)
print(response.text)

In [26]:
import requests
from bs4 import BeautifulSoup

# Get the HTML content of the S&P 500 Wikipedia page
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
response = requests.get(url)
html_content = response.text

# Parse the HTML content to extract the stock tickers and exchanges
soup = BeautifulSoup(html_content, 'html.parser')
table = soup.find('table', {'class': 'wikitable sortable'})
tickers = []
exchanges = []
for row in table.find_all('tr')[1:]:
    ticker = row.find('td').text
    ticker = ticker.replace('^', '')
    tickers.append(ticker)
    exchange = row.find_all('td')[1].text
    exchanges.append(exchange)

# Create a Python list of individual stock tickers
ticker_list = []
for ticker in tickers:
    ticker_list.append(ticker)
print(ticker_list)

# Create a Python list of stock tickers and exchanges
ticker_exchange_list = []
for i in range(len(tickers)):
    ticker_exchange_list.append((tickers[i], exchanges[i]))
print(ticker_exchange_list)

# Print the total number of tickers in the list
print(f'Total number of tickers: {len(ticker_list)}')

['MMM\n', 'AOS\n', 'ABT\n', 'ABBV\n', 'ACN\n', 'ADBE\n', 'AMD\n', 'AES\n', 'AFL\n', 'A\n', 'APD\n', 'ABNB\n', 'AKAM\n', 'ALB\n', 'ARE\n', 'ALGN\n', 'ALLE\n', 'LNT\n', 'ALL\n', 'GOOGL\n', 'GOOG\n', 'MO\n', 'AMZN\n', 'AMCR\n', 'AEE\n', 'AAL\n', 'AEP\n', 'AXP\n', 'AIG\n', 'AMT\n', 'AWK\n', 'AMP\n', 'AME\n', 'AMGN\n', 'APH\n', 'ADI\n', 'ANSS\n', 'AON\n', 'APA\n', 'AAPL\n', 'AMAT\n', 'APTV\n', 'ACGL\n', 'ADM\n', 'ANET\n', 'AJG\n', 'AIZ\n', 'T\n', 'ATO\n', 'ADSK\n', 'ADP\n', 'AZO\n', 'AVB\n', 'AVY\n', 'AXON\n', 'BKR\n', 'BALL\n', 'BAC\n', 'BK\n', 'BBWI\n', 'BAX\n', 'BDX\n', 'BRK.B\n', 'BBY\n', 'BIO\n', 'TECH\n', 'BIIB\n', 'BLK\n', 'BX\n', 'BA\n', 'BKNG\n', 'BWA\n', 'BXP\n', 'BSX\n', 'BMY\n', 'AVGO\n', 'BR\n', 'BRO\n', 'BF.B\n', 'BLDR\n', 'BG\n', 'CDNS\n', 'CZR\n', 'CPT\n', 'CPB\n', 'COF\n', 'CAH\n', 'KMX\n', 'CCL\n', 'CARR\n', 'CTLT\n', 'CAT\n', 'CBOE\n', 'CBRE\n', 'CDW\n', 'CE\n', 'COR\n', 'CNC\n', 'CNP\n', 'CF\n', 'CHRW\n', 'CRL\n', 'SCHW\n', 'CHTR\n', 'CVX\n', 'CMG\n', 'CB\n', 'CHD\n', 'C

In [30]:
question = "write python codes to download individuals stock tickers that is made up in  S&P 500. dont use ^GSPC. remove\n after every ticker and create a python list. also create another list with stock tickers and the stock exchange code in which it is traded. print the total number of tickers in the list. make sure you remove \n after each ticker"

response = chat.send_message(prompt.format(question), **parameters)
print(response.text)

 ```python
import requests
from bs4 import BeautifulSoup

# Get the HTML content of the S&P 500 Wikipedia page
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
response = requests.get(url)
html_content = response.text

# Parse the HTML content to extract the stock tickers and stock exchange codes
soup = BeautifulSoup(html_content, 'html.parser')
table = soup.find('table', {'class': 'wikitable sortable'})
tickers = []
stock_exchange_codes = []
for row in table.find_all('tr')[1:]:
    ticker = row.find('td').text
    ticker = ticker.replace('^', '')
    ticker = ticker.replace(' ', '')
    tickers.append(ticker)
    stock_exchange_code = row.find_all('td')[2].text
    stock_exchange_codes.append(stock_exchange_code)

# Create a Python list of individual stock tickers
ticker_list = []
for ticker in tickers:
    ticker_list.append(ticker)

# Create a Python list of stock tickers and stock exchange codes
ticker_stock_exchange_code_list = []
for i in range(len(tickers)):
   

In [31]:
import requests
from bs4 import BeautifulSoup

# Get the HTML content of the S&P 500 Wikipedia page
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
response = requests.get(url)
html_content = response.text

# Parse the HTML content to extract the stock tickers and stock exchange codes
soup = BeautifulSoup(html_content, 'html.parser')
table = soup.find('table', {'class': 'wikitable sortable'})
tickers = []
stock_exchange_codes = []
for row in table.find_all('tr')[1:]:
    ticker = row.find('td').text
    ticker = ticker.replace('^', '')
    ticker = ticker.replace(' ', '')
    tickers.append(ticker)
    stock_exchange_code = row.find_all('td')[2].text
    stock_exchange_codes.append(stock_exchange_code)

# Create a Python list of individual stock tickers
ticker_list = []
for ticker in tickers:
    ticker_list.append(ticker)
print(ticker_list)

# Create a Python list of stock tickers and stock exchange codes
ticker_stock_exchange_code_list = []
for i in range(len(tickers)):
    ticker_stock_exchange_code_list.append((tickers[i], stock_exchange_codes[i]))
print(ticker_stock_exchange_code_list)
# Print the total number of tickers in the list
print(f'Total number of tickers: {len(ticker_list)}')

['MMM\n', 'AOS\n', 'ABT\n', 'ABBV\n', 'ACN\n', 'ADBE\n', 'AMD\n', 'AES\n', 'AFL\n', 'A\n', 'APD\n', 'ABNB\n', 'AKAM\n', 'ALB\n', 'ARE\n', 'ALGN\n', 'ALLE\n', 'LNT\n', 'ALL\n', 'GOOGL\n', 'GOOG\n', 'MO\n', 'AMZN\n', 'AMCR\n', 'AEE\n', 'AAL\n', 'AEP\n', 'AXP\n', 'AIG\n', 'AMT\n', 'AWK\n', 'AMP\n', 'AME\n', 'AMGN\n', 'APH\n', 'ADI\n', 'ANSS\n', 'AON\n', 'APA\n', 'AAPL\n', 'AMAT\n', 'APTV\n', 'ACGL\n', 'ADM\n', 'ANET\n', 'AJG\n', 'AIZ\n', 'T\n', 'ATO\n', 'ADSK\n', 'ADP\n', 'AZO\n', 'AVB\n', 'AVY\n', 'AXON\n', 'BKR\n', 'BALL\n', 'BAC\n', 'BK\n', 'BBWI\n', 'BAX\n', 'BDX\n', 'BRK.B\n', 'BBY\n', 'BIO\n', 'TECH\n', 'BIIB\n', 'BLK\n', 'BX\n', 'BA\n', 'BKNG\n', 'BWA\n', 'BXP\n', 'BSX\n', 'BMY\n', 'AVGO\n', 'BR\n', 'BRO\n', 'BF.B\n', 'BLDR\n', 'BG\n', 'CDNS\n', 'CZR\n', 'CPT\n', 'CPB\n', 'COF\n', 'CAH\n', 'KMX\n', 'CCL\n', 'CARR\n', 'CTLT\n', 'CAT\n', 'CBOE\n', 'CBRE\n', 'CDW\n', 'CE\n', 'COR\n', 'CNC\n', 'CNP\n', 'CF\n', 'CHRW\n', 'CRL\n', 'SCHW\n', 'CHTR\n', 'CVX\n', 'CMG\n', 'CB\n', 'CHD\n', 'C

## Write test code

In [32]:
question = '''Write unit tests for the following function:

def reverse_bits(n):
  """Reverses the bits of an integer.

  Args:
    n: The integer to reverse.

  Returns:
    The reversed integer.
  """

  # Convert the integer to a binary string.
  binary_string = bin(n)[2:]

  # Reverse the binary string.
  reversed_binary_string = binary_string[::-1]

  # Convert the reversed binary string back to an integer.
  reversed_integer = int(reversed_binary_string, 2)

  return reversed_integer
'''

response = chat.send_message(prompt.format(question), **parameters)
print(response.text)

 ```python
import unittest

class TestReverseBits(unittest.TestCase):

    def test_reverse_bits(self):
        # Test case 1: Positive integer
        self.assertEqual(reverse_bits(10), 5)

        # Test case 2: Negative integer
        self.assertEqual(reverse_bits(-10), -5)

        # Test case 3: Zero
        self.assertEqual(reverse_bits(0), 0)

        # Test case 4: Maximum positive integer
        self.assertEqual(reverse_bits(2**31 - 1), 1)

        # Test case 5: Minimum negative integer
        self.assertEqual(reverse_bits(-2**31), -1)

if __name__ == '__main__':
    unittest.main()
```


In [33]:
question = '''Write a test fixture for the following class:

class TemperatureConverter:
  """A class that can be used to convert Fahrenheit to Celsius
              and visa versa."""

  def __init__(self):
    self.fahrenheit_to_celsius_factor = 5 / 9
    self.celsius_to_fahrenheit_factor = 9 / 5

  def fahrenheit_to_celsius(self, fahrenheit):
    """Converts a temperature in Fahrenheit to Celsius."""
    return (fahrenheit - 32) * self.fahrenheit_to_celsius_factor

  def celsius_to_fahrenheit(self, celsius):
    """Converts a temperature in Celsius to Fahrenheit."""
    return (celsius * self.celsius_to_fahrenheit_factor) + 32
'''

response = chat.send_message(prompt.format(question), **parameters)
print(response.text)

 ```python
import unittest

class TestTemperatureConverter(unittest.TestCase):

    def setUp(self):
        self.converter = TemperatureConverter()

    def test_fahrenheit_to_celsius(self):
        # Test case 1: Freezing point of water
        self.assertEqual(self.converter.fahrenheit_to_celsius(32), 0)

        # Test case 2: Room temperature
        self.assertEqual(self.converter.fahrenheit_to_celsius(72), 22.22)

        # Test case 3: Boiling point of water
        self.assertEqual(self.converter.fahrenheit_to_celsius(212), 100)

    def test_celsius_to_fahrenheit(self):
        # Test case 1: Freezing point of water
        self.assertEqual(self.converter.celsius_to_fahrenheit(0), 32)

        # Test case 2: Room temperature
        self.assertEqual(self.converter.celsius_to_fahrenheit(22.22), 72)

        # Test case 3: Boiling point of water
        self.assertEqual(self.converter.celsius_to_fahrenheit(100), 212)

if __name__ == '__main__':
    unittest.main()
```


## Code explanations

In [34]:
question = """
Explain the following function:

def rate_limit(max_per_minute):
    period = 60 / max_per_minute
    print("Waiting")
    while True:
        before = time.time()
        yield
        after = time.time()
        elapsed = after - before
        sleep_time = max(0, period - elapsed)
        if sleep_time > 0:
            print(".", end="")
            time.sleep(sleep_time)

"""

response = chat.send_message(prompt.format(question), **parameters)
print(response.text)

 The `rate_limit` function is a decorator that can be used to limit the number of times a function can be called within a given time period.
The function takes a single argument, `max_per_minute`, which specifies the maximum number of times the decorated function can be called per minute.
The function works by using a generator to yield control back to the caller after each call to the decorated function.
The function then calculates the time elapsed since the last call to the decorated function and sleeps for the remaining time in the current minute, if necessary.
This ensures that the decorated function is not called more than `max_per_minute` times per minute.
The function also prints a message to the console each time it sleeps, so that the user can see that the function is being rate limited.


## Adding documentation and comments

In [35]:
question = """
Rewrite the following function with proper documentation:

def rate_limit(max_per_minute):
    period = 60 / max_per_minute
    print("Waiting")
    while True:
        before = time.time()
        yield
        after = time.time()
        elapsed = after - before
        sleep_time = max(0, period - elapsed)
        if sleep_time > 0:
            print(".", end="")
            time.sleep(sleep_time)

"""

response = chat.send_message(prompt.format(question), **parameters)
print(response.text)

 ```python
def rate_limit(max_per_minute):

    """A decorator that limits the number of times a function can be called within a given time period.

    Args:
        max_per_minute: The maximum number of times the decorated function can be called per minute.

    Returns:
        A decorator function that applies the rate limit to the decorated function.
    """

    period = 60 / max_per_minute

    def decorator(func):

        def wrapper(*args, **kwargs):
            print("Waiting")
            while True:
                before = time.time()
                yield
                after = time.time()
                elapsed = after - before
                sleep_time = max(0, period - elapsed)
                if sleep_time > 0:
                    print(".", end="")
                    time.sleep(sleep_time)

        return wrapper

    return decorator
```


In [36]:
question = """
Rewrite the following function with comments:

def reverse_bits(n):

  binary_string = bin(n)[2:]
  reversed_binary_string = binary_string[::-1]
  reversed_integer = int(reversed_binary_string, 2)

  return reversed_integer
"""


response = chat.send_message(prompt.format(question), **parameters)
print(response.text)


 ```python
def reverse_bits(n):

  """Reverses the bits of an integer.

  Args:
    n: The integer to reverse.

  Returns:
    The reversed integer.
  """

  # Convert the integer to a binary string.
  binary_string = bin(n)[2:]

  # Reverse the binary string.
  reversed_binary_string = binary_string[::-1]

  # Convert the reversed binary string back to an integer.
  reversed_integer = int(reversed_binary_string, 2)

  return reversed_integer
```


## Code debugging

In [37]:
question = '''
The following function does not work.
Debug it for me and explain what the problem is:

def fahrenheit_to_celsius(fahrenheit):
  """Converts Fahrenheit to Celsius.

  Args:
    fahrenheit: A float representing the temperature in Fahrenheit.

  Returns:
    A float representing the temperature in Celsius.
  """

  celsius = fahrenheit - 32 * 5 / 9
  return celsius
'''

response = chat.send_message(prompt.format(question), **parameters)
print(response.text)

 The problem with the code is that the multiplication and division operations are not performed in the correct order.
The correct formula for converting Fahrenheit to Celsius is `celsius = (fahrenheit - 32) * 5 / 9`.
Here is the corrected code:

```python
def fahrenheit_to_celsius(fahrenheit):
  """Converts Fahrenheit to Celsius.

  Args:
    fahrenheit: A float representing the temperature in Fahrenheit.

  Returns:
    A float representing the temperature in Celsius.
  """

  celsius = (fahrenheit - 32) * 5 / 9
  return celsius
```


## Code Optimization

In [38]:
question = '''
The following function is slow,
can you optimize it for me:

def getpi():
    k = 1
    s = 0
    for i in range(1000000):
	    # even index elements are positive
        if i % 2 == 0:
            s += 4/k
        else:
		    # odd index elements are negative
            s -= 4/k
	    # denominator is odd
        k += 2

    return s
'''

response = chat.send_message(prompt.format(question), **parameters)
print(response.text)

 The function can be optimized by using the sum() function and a list comprehension.
Here is the optimized code:

```python
def getpi():
    k = 1
    s = sum([4/k if i % 2 == 0 else -4/k for i in range(1000000)])
    return s
```


## Code Conversion

In [39]:
question = '''
Convert the following Python function to JavaScript:

def reverse_bits(n):

  binary_string = bin(n)[2:]
  reversed_binary_string = binary_string[::-1]
  reversed_integer = int(reversed_binary_string, 2)

  return reversed_integer
'''

response = chat.send_message(prompt.format(question), **parameters)
print(response.text)


 ```javascript
function reverseBits(n) {
  // Convert the integer to a binary string.
  let binaryString = n.toString(2);

  // Reverse the binary string.
  let reversedBinaryString = binaryString.split("").reverse().join("");

  // Convert the reversed binary string back to an integer.
  let reversedInteger = parseInt(reversedBinaryString, 2);

  return reversedInteger;
}
```


# Run BigQuery queries from natural language

In [40]:
import vertexai
from vertexai.language_models import CodeGenerationModel
from vertexai.preview.language_models import CodeGenerationModel

vertexai.init(project="qwiklabs-gcp-01-552593a7017b", location="us-central1")
parameters = {
    "candidate_count": 1,
    "max_output_tokens": 1024,
    "temperature": 0.2
}
model = CodeGenerationModel.from_pretrained("code-bison@001")

In [41]:
prompt = '''
    Context: You write SQL Queries based on natural language inputs.
    Your queries will run on BigQuery.
    You should always generate Google Standard SQL.
    Only return the SQL statement.
    Make sure you put semicolons after each line.
    Always include the project_id in the from clause.
    Surround the full table name with back tics

    Given the following database Schema:
    {0}

    Q: Write a SQL SELECT query based on the following question: How many customers are in Virginia.
    A: SELECT COUNT(*) AS num_customers FROM `{1}.dataset_id.customers` WHERE country = 'USA' AND region = 'VA';

    Q: Write a SQL SELECT query based on the following question: {2}.
    A:
    '''

In [42]:
data_project_id = "joey-gagliardo"
dataset_id = "northwind"

from google.cloud import bigquery

client = bigquery.Client(project=PROJECT_ID)

schema_query = """SELECT
  table_catalog AS project_id,
  table_schema AS dataset_id,
  table_name AS table_name,
  ARRAY_AGG(STRUCT(
      column_name AS name,
      data_type AS type)
  ORDER BY
    ordinal_position) AS SCHEMA
FROM
  `{0}.{1}.INFORMATION_SCHEMA.COLUMNS`
GROUP BY
  table_catalog,table_schema,table_name
  """.format(data_project_id, dataset_id)

schema = list(client.query(schema_query).result())
print(schema)

[Row(('joey-gagliardo', 'northwind', 'usstates', [{'name': 'state_id', 'type': 'INT64'}, {'name': 'state_name', 'type': 'STRING(100)'}, {'name': 'state_abbr', 'type': 'STRING(2)'}, {'name': 'state_region', 'type': 'STRING(50)'}]), {'project_id': 0, 'dataset_id': 1, 'table_name': 2, 'SCHEMA': 3}), Row(('joey-gagliardo', 'northwind', 'order_details', [{'name': 'order_id', 'type': 'INT64'}, {'name': 'product_id', 'type': 'INT64'}, {'name': 'unit_price', 'type': 'FLOAT64'}, {'name': 'quantity', 'type': 'INT64'}, {'name': 'discount', 'type': 'FLOAT64'}]), {'project_id': 0, 'dataset_id': 1, 'table_name': 2, 'SCHEMA': 3}), Row(('joey-gagliardo', 'northwind', 'products', [{'name': 'product_id', 'type': 'INT64'}, {'name': 'product_name', 'type': 'STRING(40)'}, {'name': 'supplier_id', 'type': 'INT64'}, {'name': 'category_id', 'type': 'INT64'}, {'name': 'quantity_per_unit', 'type': 'STRING(20)'}, {'name': 'unit_price', 'type': 'FLOAT64'}, {'name': 'units_in_stock', 'type': 'INT64'}, {'name': 'uni

In [43]:

question = '''
How many orders were placed each year?
'''

prompt.format(schema, data_project_id, question)

"\n    Context: You write SQL Queries based on natural language inputs.\n    Your queries will run on BigQuery.\n    You should always generate Google Standard SQL.\n    Only return the SQL statement.\n    Make sure you put semicolons after each line.\n    Always include the project_id in the from clause.\n    Surround the full table name with back tics\n\n    Given the following database Schema:\n    [Row(('joey-gagliardo', 'northwind', 'usstates', [{'name': 'state_id', 'type': 'INT64'}, {'name': 'state_name', 'type': 'STRING(100)'}, {'name': 'state_abbr', 'type': 'STRING(2)'}, {'name': 'state_region', 'type': 'STRING(50)'}]), {'project_id': 0, 'dataset_id': 1, 'table_name': 2, 'SCHEMA': 3}), Row(('joey-gagliardo', 'northwind', 'order_details', [{'name': 'order_id', 'type': 'INT64'}, {'name': 'product_id', 'type': 'INT64'}, {'name': 'unit_price', 'type': 'FLOAT64'}, {'name': 'quantity', 'type': 'INT64'}, {'name': 'discount', 'type': 'FLOAT64'}]), {'project_id': 0, 'dataset_id': 1, 'ta

In [44]:
question = '''
How many orders were placed each year?
'''

response = model.predict(
    prefix = prompt.format(schema, data_project_id, question),
    **parameters
)

generated_sql = response.text.strip()
print(generated_sql)

SELECT YEAR(order_date) AS year, COUNT(*) AS num_orders
FROM `joey-gagliardo.dataset_id.orders`
GROUP BY year;


In [45]:
from google.cloud import bigquery

client = bigquery.Client(project=PROJECT_ID)
results = client.query(generated_sql).to_dataframe()
results


Forbidden: 403 Access Denied: Table joey-gagliardo:dataset_id.orders: User does not have permission to query table joey-gagliardo:dataset_id.orders, or perhaps it does not exist in location US.

Location: US
Job ID: 2b074d78-5754-44d2-a351-fefe846b768b
